In [ ]:
import pandas as pd
from pyprojroot import here
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve, confusion_matrix, f1_score

import numpy as np

In [ ]:
fig, axs = plt.subplots(figsize=(7,14), ncols=2, nrows=4, sharey=True)

loc_dict = {'bedoccupying': 0,
            'medical': 1,
            'surgical': 2,
            'critical': 3}

for path in (here() / 'data/processed/matrices/prob').glob('*-a-1.csv'):

    RESOLUTION = path.stem.split('-')[0]
    TARGET = path.stem.split('-')[1]
    MODEL = path.stem.split('-')[2]
    FS = path.stem.split('-')[3]
    HPO = path.stem.split('-')[4]

    if MODEL=='sn':
        continue
    
    pred_name = f'{RESOLUTION}-{TARGET}-{MODEL}-{FS}-{HPO}'
    true_name = f'{RESOLUTION}-{TARGET}'
    
    y_true = pd.read_csv(here() / f'data/processed/true_matrices/{true_name}.csv', index_col='Datetime')
    y_pred = pd.read_csv(here() / f'data/processed/matrices/prob/{pred_name}.csv', index_col='Datetime')

    df = pd.concat([y_true, y_pred], axis=1).dropna()
    y_true = df.iloc[:,0]
    y_pred = df.iloc[:,1]

    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    _auc = auc(fpr, tpr)
    
    auc1 = axs[loc_dict[TARGET], 1]
    auc1.plot(fpr, tpr, label=f"{path.stem} {_auc.round(2)}")
    auc1.set_xlim(0,1)
    auc1.set_ylim(0,1)

    ident = [0.0, 1.0]
    auc1.plot(ident,ident, ls='--', c='k', zorder=0, lw=.7)
    auc1.legend()
    
    sensitivities = []
    specificities = []
    f1_scores = []
    youdens_js = []
    
    for thr in thresholds:
        y_point = y_pred>thr
        # F1 
        f1_scores += [f1_score(y_true, y_point)]

        cm = confusion_matrix(y_true, y_point)

        TN = cm[0][0]
        FN = cm[1][0]
        TP = cm[1][1]
        FP = cm[0][1]

        # Sensitivity, hit rate, recall, or true positive rate
        sensitivity = TP/(TP+FN)
        sensitivities.append(sensitivity)
        
        # Specificity or true negative rate
        specificity = TN/(TN+FP)
        specificities.append(specificity)

        youdens_s = sensitivity + specificity - 1
        youdens_js.append(youdens_s)
        
    ax0 = axs[loc_dict[TARGET], 0]
    ax0.plot(sensitivities, label='sensitivity')
    ax0.plot(specificities, label='specificity')
    ax0.plot(f1_scores, label='f1')
    ax0.plot(youdens_js, label='youdens j')
    
    ax0.set_ylim([0,1])
    ax0.legend()


plt.tight_layout()
plt.savefig(here() / 'output/plots/threshold.png', dpi=300, bbox_inches='tight')